In [10]:
import pandas as pd
import json
import requests
from urllib.parse import quote
%matplotlib inline 

# Get crime data 
soql = "https://data.cityofchicago.org/resource/6zsd-86xi.json?$query="
soql += quote("SELECT community_area AS Community, count(*)/5 AS Crime_Rate, primary_type AS Primary_Type ")
soql += quote("WHERE 2007 < year AND year < 2013 ")
soql += quote("GROUP BY Community, Primary_Type LIMIT 10000")

resp = requests.get(soql).json()

crime_type = pd.DataFrame(resp)

# Get rid of the redundant decimal places 
crime_type.Crime_Rate = crime_type.Crime_Rate.astype(float)
crime_type.round(1) 

# ATTENTION HERE: problem with NaN remains unsolved!!!!
crime_type.isnull() 
crime_type.dropna(axis = 0, how = "any") 
crime_type.tail() 

# Get socioeconomics data 
socioeconomics = pd.read_csv("Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv", index_col = ['Community Area Number'])


# Merge crime data on socioeconomics data using Communty Area Number 
crime_type.rename(columns={'Community':'Community Area Number'}, inplace = True)
crime_type.set_index('Community Area Number')

# ATTENTION HERE: merge problem unsolved!!!!
mix_type = pd.DataFrame.merge(crime_type, socioeconomics, on = 'Community Area Number')
mix_type 



,Community,Crime_Rate,Primary_Type
2040,NaN,1.2,PUBLIC PEACE VIOLATION
2041,NaN,3.2,ROBBERY
2042,NaN,0.4,SEX OFFENSE
2043,NaN,44.8,THEFT
2044,NaN,1.0,WEAPONS VIOLATION
